In [66]:
#!python -m spacy download es_core_news_sm
#!python -m spacy link es_core_news_sm es

#!pip install -U gensim
#!pip install -U spacy
#!pip install nltk
#!pip install pyLDAvis

In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import plotly.express as px
from difflib import SequenceMatcher

In [2]:
# Reclamos
file_name = 'data/data_merged.csv'
data_to_process = pd.read_pickle(file_name)

In [3]:
# Filtramos solo nivel 1
data_to_process = data_to_process[data_to_process['Nivel']=='1er'].reset_index(drop=True)

In [4]:
dicionarios_reglas = {}
dicionarios_reglas['ATENCION_USUARIO'] = {}
dicionarios_reglas['ATENCION_USUARIO']['servicios'] =['96']
dicionarios_reglas['ATENCION_USUARIO']['motivos'] =['39']
dicionarios_reglas['ATENCION_USUARIO']['canal'] =[]

In [5]:
dicionarios_reglas


{'ATENCION_USUARIO': {'servicios': ['96'], 'motivos': [], 'canal': []}}

In [6]:
# Filtramos servicion de interes

# ATENCION AL USUARIO

In [10]:
data_temp = data_to_process[(data_to_process['Servicio SBS']=='96')]
data_temp.head()

,Numero reclamo,Nombre,Numero documento,Cod servicio,Cod motivo,Cod ATM,ATM error,Numero cajero,Monto,Fecha operacion,...,fecha fase 7,Servicio BN,Motivo BN,Canal Incidencia SBS,Servicio SBS,Motivo SBS,Canal Incidencia,Servicio,Motivo,Nivel
160286,002718-6551,CESAR CARRASCO BENITES,10325709,9600,150000,NaN,NaN,NaN,NaN,21/11/2018,...,2018-12-15,9600,150000,01,96,15,OFICINA,SERVICIO DE ATENCION AL USUARIO,PROBLEM. CON TARJETA DEBITO / CREDITO (RETENID...,1er
160287,002718-4863,ZAIRA CONDORI MICHUY,15450053,9600,150000,NaN,NaN,NaN,NaN,19/09/2018,...,2018-11-17,9600,150000,01,96,15,OFICINA,SERVICIO DE ATENCION AL USUARIO,PROBLEM. CON TARJETA DEBITO / CREDITO (RETENID...,1er
160288,002718-4360,JUANA AGUILAR CCENCHO,22241531,9600,150000,NaN,NaN,NaN,NaN,03/09/2018,...,2018-10-05,9600,150000,01,96,15,OFICINA,SERVICIO DE ATENCION AL USUARIO,PROBLEM. CON TARJETA DEBITO / CREDITO (RETENID...,1er
160289,001318-156,GUEVARA REBATA NORA ELVA,10013901,9600,150000,NaN,NaN,NaN,NaN,27/07/2018,...,2018-08-13,9600,150000,01,96,15,OFICINA,SERVICIO DE ATENCION AL USUARIO,PROBLEM. CON TARJETA DEBITO / CREDITO (RETENID...,1er
160290,002718-3476,FELIX DOMINGUEZ ESCUDERO,08142910,9600,150000,NaN,NaN,NaN,NaN,13/07/2018,...,2018-07-31,9600,150000,01,96,15,OFICINA,SERVICIO DE ATENCION AL USUARIO,PROBLEM. CON TARJETA DEBITO / CREDITO (RETENID...,1er


In [12]:
data_temp = data_to_process[(data_to_process['Servicio SBS']=='96')&(data_to_process['Motivo SBS']=='39')]
data_temp.head()

,Numero reclamo,Nombre,Numero documento,Cod servicio,Cod motivo,Cod ATM,ATM error,Numero cajero,Monto,Fecha operacion,...,fecha fase 7,Servicio BN,Motivo BN,Canal Incidencia SBS,Servicio SBS,Motivo SBS,Canal Incidencia,Servicio,Motivo,Nivel
166548,001318-277,AVALOS CHUMPITAZ AMELIA TERESA,08814304,9600,390000,NaN,NaN,NaN,NaN,31/12/2018,...,2019-01-31,9600,390000,01,96,39,OFICINA,SERVICIO DE ATENCION AL USUARIO,INADECUADA ATENCION AL USUARIO,1er
166549,005118-266,CIEZA RODRIGUEZ ESTHER CELINDA,41211946,9600,390000,NaN,NaN,NaN,NaN,31/12/2018,...,2019-02-08,9600,390000,01,96,39,OFICINA,SERVICIO DE ATENCION AL USUARIO,INADECUADA ATENCION AL USUARIO,1er
166550,002518-409,ESPINOZA BERROCAL JAIME EDIEE,08064474,9600,390000,NaN,NaN,NaN,NaN,29/12/2018,...,2019-01-31,9600,390000,01,96,39,OFICINA,SERVICIO DE ATENCION AL USUARIO,INADECUADA ATENCION AL USUARIO,1er
166551,002718-7738,HASSAN QUIÑONES VILLASIS,16733568,9600,390000,NaN,NaN,NaN,NaN,31/12/2018,...,2019-01-25,9600,390000,01,96,39,OFICINA,SERVICIO DE ATENCION AL USUARIO,INADECUADA ATENCION AL USUARIO,1er
166552,006118-170,VARGAS PEZUA ALICIA,41183944,9600,390000,NaN,NaN,NaN,NaN,31/12/2018,...,2019-02-05,9600,390000,01,96,39,OFICINA,SERVICIO DE ATENCION AL USUARIO,INADECUADA ATENCION AL USUARIO,1er


In [13]:
data_to_process['Servicio BN'].unique()

array(['100', '1100', '1200', '1700', '1800', '200', '2200', '2300',
       '2700', '2701', '2801', '2901', '2902', '3000', '301', '302',
       '303', '304', '400', '700', '800', '900', '9600', '9902', '9904',
       '9905'], dtype=object)

In [14]:
data_temp['Descripcion']

166548    CLIENTA MANIFIESTA QUE EL DIA 31/12/2018 SE AC...
166549    CLIENTE MENCIONA QUE SE ACERCO A LA VENTANILLA...
166550    TRANSCRITO SEGUN LIBRO DE RECLAMOS\r\n"NO QUIS...
166551    HOY DIA 31.12.2018 APROX. 11.00 AM ME APERSONE...
166552    CLIENTE INDICA QUE EL BANCO DE VILLA EL SVADOR...
                                ...                        
176483    SOLICITO Y RECLAMO QUE POR NECESIDAD HAIGA DOS...
176484    EL MOTIVO DEL RECLAMO ES SOBRE SUS PENSIÓN 65 ...
176485    LA RECLAMANTE AFIRMA QUE APROXIMA. A LAS 15.20...
176486    ME ACERQUE EL DIA 31.10.2020 PARA REALIZAR MI ...
176487    SE SUPONE QUE EL BANCO DE CHALHUAHUACHO DEBE A...
Name: Descripcion, Length: 9940, dtype: object

In [15]:
# Descripcion de hechos ()

In [79]:
data_temp['Descripcion'].values[100]

C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'CARECEN DE VENTANILLAS DE ATENCION AL PUBLICO CAJERO DEMASIADO LENTO, FALTA DE EMPATIA.\r\n\r\nINGRESADO EN LIBRO DE RESPANDO FOL. 78221'

# Modelo LDA

In [16]:
import re

import plotly.express as px
from difflib import SequenceMatcher

import nltk
nltk.download('stopwords')
#from nltk.book import *
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
# Texto a  minusculas
data_temp['texto']= data_temp['Descripcion'].str.lower()

# nan to  ''
print('Elementos nulos', len(data_temp[data_temp['texto'].isna()]))
data_temp['texto'] = data_temp['texto'].fillna('')
print('Elementos nulos', len(data_temp[data_temp['texto'].isna()]))

Elementos nulos 0
Elementos nulos 0


<ipython-input-19-0b91ea728510>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_temp['texto']= data_temp['Descripcion'].str.lower()
<ipython-input-19-0b91ea728510>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_temp['texto'] = data_temp['texto'].fillna('')


### Preparacion de palabras clave

In [20]:
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = stopwords.words('spanish')
stop_words.extend(["rt", "aun", "oe"])

exclude = string.punctuation
exclude = exclude + "¿"

lemma = SnowballStemmer('spanish')

In [21]:
print(stop_words)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

### Eliminar URL y caracteres de nueva línea

In [22]:
def remove_three_dots(list_text):
    return [re.sub(r"[a-zA-Z]+(\……|\…)$", " ", texto) for texto in list_text]

def remove_url(list_text):
    return [re.sub(r"http\S+", "", texto).strip() for texto in list_text]

def remove_breakline(list_text):
    return [re.sub('\s+', ' ', texto) for texto in list_text]

def remove_single_quotes(list_text):
    return [re.sub("\'", "", texto) for texto in list_text]

### Validando funciones de procesamiento

In [80]:
txt = data_temp['texto'].values.tolist()
print(txt[100])

carecen de ventanillas de atencion al publico cajero demasiado lento, falta de empatia.

ingresado en libro de respando fol. 78221


C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Tokenizando palabra y limpieza de texto

In [32]:
import gensim

cleared_txt = remove_url(txt)
cleared_txt = remove_three_dots(cleared_txt)
cleared_txt = remove_breakline(cleared_txt)
cleared_txt = remove_single_quotes(cleared_txt)

In [81]:
cleared_txt[100]

C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'carecen de ventanillas de atencion al publico cajero demasiado lento, falta de empatia. ingresado en libro de respando fol. 78221'

In [34]:
def texto_a_palabras(texto:str):
    for sentencia in texto:
        yield(gensim.utils.simple_preprocess(str(sentencia), deacc=True))

def texto_a_palabras2(texto:str):
    for sentencia in texto:
        yield(str(sentencia).split(" "))
        
data_palabras = list(texto_a_palabras2(cleared_txt))

print(data_palabras[:10])

[['clienta', 'manifiesta', 'que', 'el', 'dia', '31/12/2018', 'se', 'acerco', 'a', 'solicitar', 'su', 'tarjeta', 'multired', 'por', 'concepto', 'judicial', 'alimentista.', 'recibio', 'un', 'mal', 'trato', 'por', 'parte', 'del', 'personal', 'de', 'atencion', 'al', 'usuario,', 'la', 'atencion', 'de', 'la', 'ventanilla', '11', 'fue', 'pesima', 'indicandole', 'que', 'no', 'la', 'atenderia', 'y', 'que', 'espere', 'que', 'le', 'atienda', 'el', 'encargado', 'de', 'tarjetas.', 'el', 'personal', 'que', 'atiende', 'en', 'atencion', 'al', 'usuario', 'debe', 'estar', 'capacitado', 'para', 'atender', 'y', 'orientar', 'mejor', 'al', 'cliente', 'y', 'disponer', 'de', 'suficiente', 'personal', 'para', 'la', 'atencion.'], ['cliente', 'menciona', 'que', 'se', 'acerco', 'a', 'la', 'ventanilla', '1', 'para', 'realizar', 'unos', 'pagos,', 'el', 'gestor', 'sin', 'saludar', 'le', 'pgto', '"', 'que', 'va', 'hacer",', 'la', 'cliente', 'le', 'mencionó', 'que', 'hará', 'unos', 'pagos', 'de', 'detracción', 'y', 'e

### N gramas

In [35]:
bigram = gensim.models.Phrases(data_palabras, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_palabras], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [36]:
data_palabras[0]

['clienta',
 'manifiesta',
 'que',
 'el',
 'dia',
 '31/12/2018',
 'se',
 'acerco',
 'a',
 'solicitar',
 'su',
 'tarjeta',
 'multired',
 'por',
 'concepto',
 'judicial',
 'alimentista.',
 'recibio',
 'un',
 'mal',
 'trato',
 'por',
 'parte',
 'del',
 'personal',
 'de',
 'atencion',
 'al',
 'usuario,',
 'la',
 'atencion',
 'de',
 'la',
 'ventanilla',
 '11',
 'fue',
 'pesima',
 'indicandole',
 'que',
 'no',
 'la',
 'atenderia',
 'y',
 'que',
 'espere',
 'que',
 'le',
 'atienda',
 'el',
 'encargado',
 'de',
 'tarjetas.',
 'el',
 'personal',
 'que',
 'atiende',
 'en',
 'atencion',
 'al',
 'usuario',
 'debe',
 'estar',
 'capacitado',
 'para',
 'atender',
 'y',
 'orientar',
 'mejor',
 'al',
 'cliente',
 'y',
 'disponer',
 'de',
 'suficiente',
 'personal',
 'para',
 'la',
 'atencion.']

In [37]:
print(bigram_mod[data_palabras[0]])

['clienta', 'manifiesta', 'que', 'el', 'dia', '31/12/2018', 'se', 'acerco', 'a', 'solicitar', 'su', 'tarjeta', 'multired', 'por', 'concepto', 'judicial', 'alimentista.', 'recibio', 'un', 'mal', 'trato', 'por', 'parte', 'del', 'personal', 'de', 'atencion', 'al', 'usuario,', 'la', 'atencion', 'de', 'la', 'ventanilla', '11', 'fue', 'pesima', 'indicandole', 'que', 'no', 'la', 'atenderia', 'y', 'que', 'espere', 'que', 'le', 'atienda', 'el', 'encargado', 'de', 'tarjetas.', 'el', 'personal', 'que', 'atiende', 'en', 'atencion', 'al', 'usuario', 'debe', 'estar', 'capacitado', 'para', 'atender', 'y', 'orientar', 'mejor', 'al', 'cliente', 'y', 'disponer', 'de', 'suficiente', 'personal', 'para', 'la', 'atencion.']


In [38]:
print(trigram_mod[data_palabras[0]])

['clienta', 'manifiesta', 'que', 'el', 'dia', '31/12/2018', 'se', 'acerco', 'a', 'solicitar', 'su', 'tarjeta', 'multired', 'por', 'concepto', 'judicial', 'alimentista.', 'recibio', 'un', 'mal_trato', 'por', 'parte', 'del', 'personal', 'de', 'atencion', 'al', 'usuario,', 'la', 'atencion', 'de', 'la', 'ventanilla', '11', 'fue', 'pesima', 'indicandole', 'que', 'no', 'la', 'atenderia', 'y', 'que', 'espere', 'que', 'le', 'atienda', 'el', 'encargado', 'de', 'tarjetas.', 'el', 'personal', 'que', 'atiende', 'en', 'atencion', 'al', 'usuario', 'debe', 'estar', 'capacitado', 'para', 'atender', 'y', 'orientar', 'mejor', 'al', 'cliente', 'y', 'disponer', 'de', 'suficiente', 'personal', 'para', 'la', 'atencion.']


### Eliminar stoprwords, bigramas y lematizando texto

In [49]:
import spacy
spacy.prefer_gpu()

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [50]:
data_words_nostops = remove_stopwords(data_palabras)
data_words_bigrams = make_bigrams(data_words_nostops)

In [53]:
#nlp = spacy.load('es', disable=['parser', 'ner'])
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [54]:
print(data_words_bigrams[0])

['clienta', 'manifiesta', 'dia', 'acerco', 'solicitar', 'tarjeta', 'multired', 'concepto', 'judicial', 'alimentista', 'recibio', 'mal', 'trato', 'parte', 'personal', 'atencion', 'usuario', 'atencion', 'ventanilla', 'pesima', 'indicandole', 'atenderia', 'espere', 'atienda', 'encargado', 'tarjetas', 'personal', 'atiende', 'atencion', 'usuario', 'debe', 'capacitado', 'atender', 'orientar', 'mejor', 'cliente', 'disponer', 'suficiente', 'personal', 'atencion']


In [82]:
print(data_lemmatized[100])

['carecer', 'ventanilla', 'atencion', 'publico', 'cajero', 'demasiado', 'lento', 'falta', 'empatiar', 'ingresado', 'libro', 'respar']


C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Creacion de diccionario y corpus
El modelo LDA requiere dos entradas principales: corpus y diccionario

In [56]:
from gensim import corpora
 
# Diccionario
diccionario = corpora.Dictionary(data_lemmatized)

# Corpus
texto = data_lemmatized
corpus = [diccionario.doc2bow(doc) for doc in texto]

In [57]:
print(data_words_bigrams[0])
print(corpus[0])

['clienta', 'manifiesta', 'dia', 'acerco', 'solicitar', 'tarjeta', 'multired', 'concepto', 'judicial', 'alimentista', 'recibio', 'mal', 'trato', 'parte', 'personal', 'atencion', 'usuario', 'atencion', 'ventanilla', 'pesima', 'indicandole', 'atenderia', 'espere', 'atienda', 'encargado', 'tarjetas', 'personal', 'atiende', 'atencion', 'usuario', 'debe', 'capacitado', 'atender', 'orientar', 'mejor', 'cliente', 'disponer', 'suficiente', 'personal', 'atencion']
[(0, 1), (1, 3), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 2), (28, 1)]


In [58]:
[[(diccionario[id], freq) for id, freq in cp] for cp in corpus[0:1]]

[[('alimentistar', 1),
  ('atencion', 3),
  ('atender', 2),
  ('atenderio', 1),
  ('atienda', 1),
  ('capacitar', 1),
  ('cliente', 1),
  ('concepto', 1),
  ('diar', 1),
  ('disponer', 1),
  ('encargado', 1),
  ('esperar', 1),
  ('indicar él', 1),
  ('judicial', 1),
  ('mal', 1),
  ('manifiesto', 1),
  ('mejor', 1),
  ('multired', 1),
  ('orientar', 1),
  ('parte', 1),
  ('personal', 3),
  ('pesimo', 1),
  ('recibio', 1),
  ('solicitar', 1),
  ('suficiente', 1),
  ('tarjeta', 2),
  ('tratar', 1),
  ('usuario', 2),
  ('ventanillo', 1)]]

In [59]:
print(len(list(diccionario.values())))

21453


In [60]:
list(diccionario.values())

['alimentistar',
 'atencion',
 'atender',
 'atenderio',
 'atienda',
 'capacitar',
 'cliente',
 'concepto',
 'diar',
 'disponer',
 'encargado',
 'esperar',
 'indicar él',
 'judicial',
 'mal',
 'manifiesto',
 'mejor',
 'multired',
 'orientar',
 'parte',
 'personal',
 'pesimo',
 'recibio',
 'solicitar',
 'suficiente',
 'tarjeta',
 'tratar',
 'usuario',
 'ventanillo',
 'acerco',
 'agencio',
 'atención',
 'caja',
 'cajo',
 'cola',
 'detracción',
 'hacer',
 'mando',
 'mencionar',
 'molesto',
 'nego',
 'ningun',
 'pago',
 'primera_vez',
 'rapido',
 'realizar',
 'suceder',
 'trabajador',
 'ventanilla',
 'contar',
 'falta',
 'libro',
 'querer',
 'segun',
 'sistema',
 'transcrito',
 'acepto',
 'acercar',
 'administrador',
 'ahorro',
 'alimento',
 'apersonar',
 'aprox',
 'avanza',
 'banco',
 'celular',
 'contestar',
 'decir',
 'decirme',
 'deposito',
 'disculpa',
 'efectue',
 'embargo',
 'error',
 'forma',
 'foto',
 'hablar',
 'hoy',
 'importante',
 'institución',
 'llamar',
 'machado',
 'malo',


### Modelo LDA

In [74]:
# Correr y entrenar el modelo LDA sobre la matriz de términos.
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=10,
                                   id2word = diccionario,
                                   random_state=100,
                                   passes=10,
                                   update_every=1,
                                   chunksize=100,
                                   alpha='auto',
                                   per_word_topics=True)

C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Lista de temas del modelo LDA

In [75]:
for idx, topic in lda_model.print_topics():
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.043*"banco" + 0.035*"atención" + 0.034*"día" + 0.029*"usuario" + 0.018*"hora" + 0.018*"servicio" + 0.016*"salir" + 0.014*"tiempo" + 0.013*"distrito" + 0.011*"llamar"

Topic: 1 Word: 0.048*"hacer" + 0.042*"decir" + 0.023*"reclamo" + 0.023*"dar" + 0.020*"querer" + 0.014*"señor" + 0.012*"manera" + 0.011*"pedir" + 0.010*"mal" + 0.010*"luego"

Topic: 2 Word: 0.055*"sol" + 0.053*"dinero" + 0.039*"cajero" + 0.034*"pagar" + 0.033*"cobrar" + 0.032*"bono" + 0.027*"monto" + 0.023*"retirar" + 0.021*"srta" + 0.020*"diar"

Topic: 3 Word: 0.049*"cheque" + 0.046*"cerrar" + 0.045*"requisito" + 0.025*"sunat" + 0.018*"empleado" + 0.018*"principal" + 0.017*"identificar" + 0.014*"devolucion" + 0.014*"irrespetuoso" + 0.013*"agosto"

Topic: 4 Word: 0.095*"celular" + 0.049*"deposito" + 0.043*"funcionario" + 0.029*"apertura" + 0.025*"ayudar" + 0.024*"carta" + 0.020*"aviso" + 0.020*"acudir" + 0.017*"alimento" + 0.015*"cartar"

Topic: 5 Word: 0.227*"tarjeta" + 0.049*"clave" + 0.046*"duplicado" +

C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Visualizacion de topicos LDA

In [76]:
#import warnings
#import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#warnings.filterwarnings('ignore')

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model,corpus,diccionario)
vis

C:\Users\johnk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.262028  0.056803       1        1  26.658706
9      0.169080 -0.352152       2        1  19.535032
6      0.215773  0.186493       3        1  17.015291
0      0.145179  0.152554       4        1  14.487647
2     -0.009155 -0.165863       5        1   9.521288
7     -0.147634  0.022913       6        1   3.651775
8     -0.140598  0.021044       7        1   3.503685
5     -0.152125  0.025483       8        1   2.588167
3     -0.171462  0.026389       9        1   1.576747
4     -0.171087  0.026336      10        1   1.461663, topic_info=           Term         Freq        Total Category  logprob  loglift
25      tarjeta  2079.000000  2079.000000  Default  30.0000  30.0000
167     persona  4273.000000  4273.000000  Default  29.0000  29.0000
45     realizar  3826.000000  3826.000000  Default  28.0000  28.0000
36        hacer  7079.000000  7079.000000  Default  27.0000  27.0000
2       atender  3310.000000  3310.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1184   reclamos    40.289510    41.166811  Topic10  -4.8547   4.2041
2348   señalado    35.521201    36.398402  Topic10  -4.9807   4.2012
2800      algun    32.966788    33.843868  Topic10  -5.0553   4.1993
4233   movistar    31.358289    32.235295  Topic10  -5.1053   4.1980
3294  dirección    29.886658    30.764143  Topic10  -5.1534   4.1967

[365 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5584      4  0.996902      abril
57        1  0.024176    acercar
57        2  0.973095    acercar
2221     10  0.980500  acercarme
29        2  0.998541     acerco
...     ...       ...        ...
820       2  0.998548  verificar
537       1  0.999160        vez
648       9  0.980011   vigencia
94        3  0.998707  vigilante
1717      5  0.998434        voz

[373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 7, 1, 3, 8, 9, 6, 4, 5])